<a href="https://colab.research.google.com/github/shiragelb/NCC-Statistical-Reports/blob/main/drive-files-uploding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()



,file_name,file_path,file_id,file_url
0,content.docx,2015/content.docx,1ZfhxSBXj-cWQpMGNGkX7vwnAd-wKsylU,https://drive.google.com/file/d/1ZfhxSBXj-cWQp...
1,15.doc,2015/15.doc,1WAk_eRJP12NzRxQzVSN54lr-w23qf5OW,https://drive.google.com/file/d/1WAk_eRJP12NzR...
2,14.docx,2015/14.docx,1X7bK7gl9UibMO0iLj0qxMX7d0nX0PJO-,https://drive.google.com/file/d/1X7bK7gl9UibMO...
3,13.doc,2015/13.doc,1xN6fxEL03XSO3lX2e2WkuhRuGxoQu_KI,https://drive.google.com/file/d/1xN6fxEL03XSO3...
4,12.doc,2015/12.doc,1mJmtlIPTXz-eVxaothuTkMfVwqo0k3q4,https://drive.google.com/file/d/1mJmtlIPTXz-eV...


In [ ]:
##getting local id of the file
!pip install python-docx
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

from docx import Document

# After downloading the file
##sainty check
# path = get_chapter_file(14, "2015")   # downloads "03.docx" from the 2022 folder
# print("Local file:", path)

# if path and path.endswith(".docx"):
#     doc = Document(path)
#     full_text = "\n".join([p.text for p in doc.paragraphs if p.text.strip() != ""])

#     # Split into sentences (naive split by period/full stop)
#     sentences = full_text.replace("\n", " ").split(".")

#     # Print first few sentences
#     preview_count = 3
#     preview_sentences = [s.strip() for s in sentences if s.strip()][:preview_count]
#     print("\nPreview of content:")
#     for i, s in enumerate(preview_sentences, 1):
#         print(f"{i}. {s}.")


⬇️ Download 100%.
✅ Saved to /content/2015/14.docx
Local file: /content/2015/14.docx

Preview of content:
1. 14 ילדים במצבי סיכון ומצוקה מבוא פרק זה, המוקדש לילדים במצבי סיכון ומצוקה, הוא אחד הפרקים המורכבים בשנתון.
2. ההגדרה מיהו ילד בסיכון או במצוקה תלויה בסביבה, בחברה, בתקופה ובמגדיר.
3. לא רק זאת, קשה לאמוד את היקף התופעות הקשורות לילדים במצבי סיכון ומצוקה, שכן לפי טבען לא תמיד הן חשופות או מדווחות, והמקרים הידועים אינם אלא קצה הקרחון.


In [ ]:
#a code that downloads the files to a directory called content/reports
import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=range(2001, 2025), chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            match = df_files[
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths
download_all_chapters()
